In [4]:
import pandas as pd
import numpy as np
import networkx as nx
import json

In [10]:
path = 'data/twibot-20/twibot20.json'

f = open(path)
df = json.load(f)

In [11]:
df

[{'ID': '17461978',
  'profile': {'id': '17461978 ',
   'id_str': '17461978 ',
   'name': 'SHAQ ',
   'screen_name': 'SHAQ ',
   'location': 'Orlando, FL ',
   'profile_location': "{'id': '55b4f9e5c516e0b6', 'url': 'https://api.twitter.com/1.1/geo/id/55b4f9e5c516e0b6.json', 'place_type': 'unknown', 'name': 'Orlando, FL', 'full_name': 'Orlando, FL', 'country_code': '', 'country': '', 'contained_within': [], 'bounding_box': None, 'attributes': {}} ",
   'description': 'VERY QUOTATIOUS, I PERFORM RANDOM ACTS OF SHAQNESS ',
   'url': 'https://t.co/7hsiK8cCKW ',
   'entities': "{'url': {'urls': [{'url': 'https://t.co/7hsiK8cCKW', 'expanded_url': 'http://www.ShaqFuRadio.com', 'display_url': 'ShaqFuRadio.com', 'indices': [0, 23]}]}, 'description': {'urls': []}} ",
   'protected': 'False ',
   'followers_count': '15349596 ',
   'friends_count': '692 ',
   'listed_count': '45568 ',
   'created_at': 'Tue Nov 18 10:27:25 +0000 2008 ',
   'favourites_count': '142 ',
   'utc_offset': 'None ',
   't